In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
def print_to_file(value, name):
    print(name, value)
    with open(name, 'w') as file:
        file.write(str(value))

# Рекомендательные системы

***Описание задачи***

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать **recall@k** и **precision@k.**

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

***Входные данные***

Вам дается две выборки с пользовательскими сессиями - **id-**шниками просмотренных и **id**-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: **id** просмотренных товаров через **,** затем идёт **;** после чего следуют **id** купленных товаров (если такие имеются), разделённые запятой. Например, **1,2,3,4;** или **1,2,3,4;5,6.**

Гарантируется, что среди **id** купленных товаров все различные.

In [3]:
def str_to_ar(string):
    return list(map(int, string.split(','))) if len(string) != 0 else None

def get_improved_df(filepath):
    df = pd.read_csv(filepath, sep = ';', names = ['viewed','buying'],
                    converters = {0:str_to_ar, 1:str_to_ar})
    print(df.head(10))
    return df

In [4]:
df_train = get_improved_df('data/coursera_sessions_train.txt')

                                              viewed        buying
0                                 [0, 1, 2, 3, 4, 5]          None
1                      [9, 10, 11, 9, 11, 12, 9, 11]          None
2                           [16, 17, 18, 19, 20, 21]          None
3                               [24, 25, 26, 27, 24]          None
4   [34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]          None
5                                               [42]          None
6                                       [47, 48, 49]          None
7  [59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...  [67, 60, 63]
8                                   [71, 72, 73, 74]          None
9                                       [76, 77, 78]          None


In [5]:
df_test = get_improved_df('data/coursera_sessions_test.txt')

                                         viewed    buying
0                                     [6, 7, 8]      None
1                                  [13, 14, 15]      None
2                                      [22, 23]      None
3                      [28, 29, 30, 31, 32, 33]      None
4                                      [40, 41]      None
5              [43, 44, 43, 45, 43, 45, 43, 46]      None
6  [50, 51, 47, 52, 49, 53, 54, 55, 56, 57, 58]      None
7      [63, 68, 69, 70, 66, 61, 59, 61, 66, 68]  [66, 63]
8                                          [75]      None
9                          [79, 80, 81, 82, 83]      None


***Важно:***

-   Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
-   Если товар не встречался в обучающей выборке, его популярность равна 0.
-   Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
-   Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и **k** в **recall@k** / **precision@k**.

***Задание***

1.  На обучении постройте частоты появления **id** в просмотренных и в купленных (**id** может несколько раз появляться в просмотренных, все появления надо учитывать)

In [6]:
def get_counter(series):
    lst = series.values
    all_values = []
    for sublist in lst:
        for item in sublist:
            all_values.append(item)
    cnt = Counter(all_values)
    return cnt

In [7]:
def get_num(series):
    # Формируем список всех встречающихся ИД в наборе
    lst = series.values
    all_values = []
    for sublist in lst:
        for item in sublist:
            all_values.append(item)
    # Считаем сколько раз встретился каждый ИД в наборе
    freq = np.bincount(all_values)
    ii = np.nonzero(freq)[0]
    # Возвращаем список - (ИД - количество раз сколько он встретился в списке)
    dic = {}
    for id_, freq_ in zip(ii,freq[ii]):
        dic[id_] = freq_
    return dic

In [8]:
cnt_train_viewed = get_counter(df_train.viewed)
cnt_train_buying = get_counter(df_train.buying.dropna())

In [9]:
num_train_viewed = get_num(df_train.viewed)
num_train_buying = get_num(df_train.buying.dropna())

2.Реализуйте два алгоритма рекомендаций:

* сортировка просмотренных **id** по популярности (частота появления в просмотренных),
* сортировка просмотренных **id** по покупаемости (частота появления в покупках).

In [10]:
def get_recommend_ids(viewed, cnt, k):
    viewed = list(set(viewed))
    lenght = min(k, len(viewed))
    pos = range(len(viewed))
    freq = list(cnt[id_] if id_ in cnt.keys() else 0 for id_ in viewed)
    d = {'id': viewed, 'pos': pos, 'freq': freq}
    df = pd.DataFrame(data = d).sort_values(['freq', 'pos'], ascending=[False, True])   
    return set(df['id'][:lenght].tolist())

3. Для данных алгоритмов выпишите через пробел **AverageRecall@1**, **AveragePrecision@1**, **AverageRecall@5**, **AveragePrecision@5** на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

In [11]:
def metric(viewed, buying, cnt, algo, k):
    recoms = get_recommend_ids(viewed, cnt, k)
    summ = 0
    for i in buying:
        if i in recoms:
            summ+=1
    if algo == 'precision':
        return summ / k
    elif algo == 'recall':
        return summ / len(buying)
    else:
        return None

In [12]:
def get_average_metric(df, cnt, algo, k):
    metric_vals = []
    for session in df.values:
        metric_vals.append(metric(session[0], session[1], cnt, algo, k))
    return sum(metric_vals) / len(metric_vals)

In [13]:
def get_metrics(df, cnt):
    return (get_average_metric(df, cnt, 'recall', 1), 
           get_average_metric(df, cnt, 'precision', 1),
           get_average_metric(df, cnt, 'recall', 5),
           get_average_metric(df, cnt, 'precision', 5))

In [14]:
def convert_for_answer(ar):
    return ' '.join(list(str(round(i, 2)) for i in ar))    

In [15]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [16]:
print_to_file(convert_for_answer(get_metrics(df_train, cnt_train_viewed)), '1-cnt.txt')
print_to_file(convert_for_answer(get_metrics(df_test, cnt_train_viewed)), '2-cnt.txt')
print_to_file(convert_for_answer(get_metrics(df_train, cnt_train_buying)), '3-cnt.txt')
print_to_file(convert_for_answer(get_metrics(df_test, cnt_train_buying)), '4-cnt.txt')

1-cnt.txt 0.44 0.51 0.82 0.21
2-cnt.txt 0.42 0.48 0.8 0.2
3-cnt.txt 0.68 0.79 0.93 0.25
4-cnt.txt 0.41 0.47 0.79 0.2


In [17]:
print_to_file(convert_for_answer(get_metrics(df_train, num_train_viewed)), '1-num.txt')
print_to_file(convert_for_answer(get_metrics(df_test, num_train_viewed)), '2-num.txt')
print_to_file(convert_for_answer(get_metrics(df_train, num_train_buying)), '3-num.txt')
print_to_file(convert_for_answer(get_metrics(df_test, num_train_buying)), '4-num.txt')

1-num.txt 0.44 0.51 0.82 0.21
2-num.txt 0.42 0.48 0.8 0.2
3-num.txt 0.68 0.79 0.93 0.25
4-num.txt 0.41 0.47 0.79 0.2


Грейдер принимает только первые три ответа - четвёртый некорректный. В четвёртом должны быть такие метрики:
* 0.46 0.53 0.82 0.21 